In [ ]:
import requests
from urllib.parse import urlencode
import pandas as pd
import urllib
import json
from tqdm import tqdm
import re

В ячейке ниже можно получить файл с яндекс диска без предварительных скачиваний \
Настоятельно рекомендую пользоваться таким форматом, чтобы впоследствии любой код можно было запускать без предварительного поиска и локального скачивания файлов.

In [ ]:
folder_url = 'YANDEX DISK FOLDER LINK' #ссылка на папку
file_url = 'FILE NAME' #тут должно быть имя файла
url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download' + '?public_key=' + urllib.parse.quote(folder_url) + '&path=/' + urllib.parse.quote(file_url)

r = requests.get(url)
h = json.loads(r.text)['href']
df = pd.read_excel(h)

План работы следующий:

- составляю общий плоский список всех аффилиаций
- рассматриваю варианты их сокращения до аффилаций
- создаю словарь сокращений

In [ ]:
affiliations = []
for i in df.columns:
    if 'affiliation' in i: #в наших данных все колонки, связанные с информацией об аффилиациях содержат слово "affiliation", поэтому колонки фильтруются по этому признаку
        affiliations.append(i)
affiliations

In [ ]:
#создаю датасет сугубо из колонок с информацией по аффилиациям
af = pd.DataFrame()
for i in affiliations:
        af[i] = df[i]
af.head()

,author_1_affiliation_id,author_1_ru affiliation,author_1_eng affiliation,author_2_affiliation_id,author_2_ru affiliation,author_2_eng affiliation,author_3_affiliation_id,author_3_ru affiliation,author_3_eng affiliation,author_4_affiliation_id,...,author_5_eng affiliation,author_6_affiliation_id,author_6_ru affiliation,author_6_eng affiliation,author_7_affiliation_id,author_7_ru affiliation,author_7_eng affiliation,author_8_affiliation_id,author_8_ru affiliation,author_8_eng affiliation
0,"[7113, 7113]",['Белгородский государственный технологический...,['Belgorod State Technological University name...,"[364, none]",['Новомосковский институт РХТУ им. Д.И. Мендел...,none,"[none, none]",none,['Novomoskovsk Institute RCTU named after D.I....,"[none, none]",...,none,"[none, none]",none,none,"[none, none]",none,none,"[none, none]",none,none
1,"[none, none]",none,none,"[none, none]",none,none,"[none, none]",none,none,"[none, none]",...,none,"[none, none]",none,none,"[none, none]",none,none,"[none, none]",none,none
2,"[340, none]",['Белгородский государственный университет'],none,"[none, none]",none,none,"[none, none]",none,none,"[none, none]",...,none,"[none, none]",none,none,"[none, none]",none,none,"[none, none]",none,none
3,"[210, none]",['Кубанский государственный университет'],none,"[none, none]",none,none,"[none, none]",none,none,"[none, none]",...,none,"[none, none]",none,none,"[none, none]",none,none,"[none, none]",none,none
4,"[2541, none]",['МГУ им. М.В. Ломоносова?'],none,"[none, none]",none,none,"[none, none]",none,none,"[none, none]",...,none,"[none, none]",none,none,"[none, none]",none,none,"[none, none]",none,none


# Шаг 0: Предварительная очистка данных

- Чистим id аффилиаций от всех ненужных знаков
- При помощи регулярных выражений оставляю только числовые значения
- получившиеся после регулярного выражения пустые поля заполняю 'none'

In [ ]:
ids = [a for a in affiliations if 'id' in a] # список колонок, содержащих информацию об аффилиациях, в которых содержится информация об ID
for a in ids
        for m in tqdm(range(len(af[a]))):
            value = af[a][m].split(',') # на случай множественных аффилиаций, перечисленных через запятую: value - список всех элементов в ячейке, разделенных запятой

            new_value = []
            for v in value:
                try:
                    v = re.sub("[^0-9]", "", v) # для каждого элемента в списке value удаляются нечисловые значения
                    new_value.append(v)
                except:
                    v = v.strip("'").strip("[").strip("]")
                    new_value.append(v)

            for i in range(len(new_value)):
                if new_value[i] == '': #если значение пустое, присуждаем 'none'
                    new_value[i] = 'none'

            af[a][m] = new_value

Та же обработка применяется к названиям аффилиаций

In [ ]:
af_train = af.copy()

In [ ]:
affs = [a for a in affiliations if 'id' not in a] # выделяю колонки, содержащие названия аффилиаций
for a in affs:
        for m in tqdm(range(len(af_train[a]))):
            if "', '" in af_train[a][m]: # проверяю, есть ли внутри списка запятая между элементами, чтобы определить длину списка
                value = af_train[a][m].split("', '")

            else:
                value = [af_train[a][m]]

            for v in range(len(value)):
                    value[v] = value[v].strip("[").strip("'").strip("]").strip("'") #очищаю названия аффилиаций от нежеланных спец. символов

            af_train[a][m] = value

In [ ]:
rus = [a for a in affiliations if 'ru' in a] #только колонки с информацией об аффилиациях на русском
rus_af = []
for i in rus:
    rus_af.append(af_train[i].to_list())
rus_af = sum(rus_af, []) #объединенный список всех аффилиаций на русском языке (всех значений колонок из rus)

In [ ]:
uniq_rus_af = list(set(rus_af))
pd.Series(uniq_rus_af).to_excel('all_affs_ru.xlsx')

In [ ]:
ru_to_id = {} #инициализирую словарь, связывающий русские названия аффилиаций с их ID
for i in uniq_rus_af:
    ru_to_id[i] = []

ru_to_eng = {} #инициализирую словарь, связывающий русские названия аффилиаций с их англоязычными названиями
for i in uniq_rus_af:
    ru_to_eng[i] = []

# Шаг 1: Работа с ID аффилиаций

Сначала создаю словарь, где ключи это аффилиации на русском, а их значения - списки, в которые будут добавляться ID

In [ ]:
for num in range(1,9):
    for i in tqdm(range(len(af_train))):
        for l in range(len(af_train['author_{}_ru affiliation'.format(num)][i])):
            ru_to_id[af_train['author_{}_ru affiliation'.format(num)][i][l]].append(
                af_train['author_{}_affiliation_id'.format(num)][i][l]
            )

In [ ]:
for i in ru_to_id.keys():
    ru_to_id[i] = list(set(ru_to_id[i]))

Теперь в словаре ru_to_id хранится для каждого русскоязычного названия аффилиации перечень ID с которыми оно встречалось\
Это позволяет:
- выявить ошибки в данных (аффилиации с несколькими ID)
- выявить разные написания одной и той же аффилиации за счет общности ID

In [ ]:
# проверяем, что длина всех значений в словаре не больше 1 (в противном случае аффилиация имеет несколько ID)
for i in ru_to_id.keys():
    if len(ru_to_id[i])>1:
        try:
            ru_to_id[i].remove('none')
        except:
            print([i, ru_to_id[i]])

['ГАОУ ВО ЛО «Ленинградский государственный университет имени А.С. Пушкина»', ['2149', '190']]


In [ ]:
# для ручного анализа выводятся аффилиации без ID
# на этом рано останавливаться, следует проверить, не имеют ли эти аффилиации общих переводов / ID английского перевода
# на текущий момент 5000 аффилиаций из 18 000 не имеют ID

counter = 0
for i in ru_to_id.keys():
    if ru_to_id[i] == ['none']:
        counter +=1
        print([i, ru_to_id[i]])
print(counter)

#Шаг 2: Работа с английским переводом аффилиаций

В словарь ru_to_eng присоединяются списком значения англоязычных переводов русских названий аффилиации

In [ ]:
for num in range(1,9):
    for i in range(len(af_train)):
        for l in range(len(af_train['author_{}_ru affiliation'.format(num)][i])):
            try:
                ru_to_eng[af_train['author_{}_ru affiliation'.format(num)][i][l]].append(
                   af_train['author_{}_eng affiliation'.format(num)][i][l])
            except:
                print('По-русски:', af_train['author_{}_ru affiliation'.format(num)][i], '\n',
                      'По-английски:', af_train['author_{}_eng affiliation'.format(num)][i], '\n')

In [ ]:
for i in ru_to_eng.keys():
    ru_to_eng[i] = list(set(ru_to_eng[i])) #оставляем только уникальные значения
for i in ru_to_eng.keys():
    if ru_to_eng[i] == []:
        ru_to_eng[i] = ['none'] #пропуски заполняются none

Для удобства создаем новый датафрейм, содержащий колонки\
ru_aff - русскоязычное написание аффилиаций (ключи словаря ru_to_id)\
id - соответствующие им значения id в словаре ru_to_id\
eng_aff - значения словаря ru_to_eng по ключу ru_aff

In [ ]:
af_df = pd.DataFrame()
af_df['ru_aff'] = list(ru_to_id.keys())
af_df['id'] = [ru_to_id[i] for i in ru_to_id.keys()]
af_df['eng_aff'] = [ru_to_eng[i] for i in ru_to_id.keys()]

In [ ]:
#удостоверяемся, что у каждой аффилиации только один id
for i in range(len(af_df['id'])):
    try:
        assert len(af_df['id'][i]) == 1
    except:
        print(af_df['id'][i], af_df['ru_aff'][i])

In [ ]:
#проверяем длину списка аффилиаций для каждого автора
#если список уникальных id аффилиаций автора длиной > 1 и в этом списке есть 'none' - удаляем 'none', оставляя только не-null значение
for num in range(1,9):
    for i in tqdm(range(len(af_train))):
        af_train['author_{}_affiliation_id'.format(num)][i] = list(set(af_train['author_{}_affiliation_id'.format(num)][i]))
        if len(af_train['author_{}_affiliation_id'.format(num)][i])>1:
            for m in af_train['author_{}_affiliation_id'.format(num)][i]:
                if m == 'none':
                    af_train['author_{}_affiliation_id'.format(num)][i].remove(m)

# Шаг 3: Создаем новые id для неидентифицированных аффилиаций

In [ ]:
#придаем id аффилиаций с разными названиями единое, заранее выбранное вручную для решения проблемы дедупликации
#все названия аффилиаций с одним и тем же id были собраны в файл, на основании которого были вручную отобраны корректные наименования институций
sev_names = pd.read_excel('id_to_rus_all.xlsx')
for num in range(1,8):
    for i in tqdm(range(len(af_train))):
        for f in range(len(sev_names)):
            try:
                if sev_names['id'][f] == int(af_train['author_{}_affiliation_id'.format(num)][i][0]):
                    af_train['author_{}_ru affiliation'.format(num)][i] = sev_names['vals'][f]
                    af_train['author_{}_affiliation_id'.format(num)][i] = [str(sev_names['id'][f])]
            except:
                pass

In [ ]:
to_create = [] # список аффилиаций без id
for num in range(1,9):
    for i in range(len(work_df)):
        if work_df['author_{}_affiliation_id'.
                   format(num)][i][0]=='none' and work_df['author_{}_ru affiliation'.
                                                          format(num)][i] != ['none']:
            to_create.append(work_df['author_{}_ru affiliation'.
                                                          format(num)][i])

In [ ]:
# создаем датафрейм со всеми неидентифицированными аффилиациями, по дефолту заполняем их новый id как 0 (в отличие от none при отсутствии аффилиации)
transfs = pd.DataFrame()
transfs['to_transform'] = list(to_create)
transfs['new_name'] = ['0']*len(transfs)
transfs['id'] = ['0']*len(transfs)

По кодовым словам выделяем наиболее частотные аффилиации и добавляем верный ID к их неидентифицированным вариациям

In [ ]:
#ранхигс
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'при президенте' in transfs['to_transform'][i][0].lower() or 'ранхигс' in transfs['to_transform'][i][0].lower() or 'ранх и гс' in transfs['to_transform'][i][0].lower() or 'хигс' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'РАНХиГС при Президенте РФ'
            transfs['id'][i] = '7082'
#вшэ
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'вшэ' in transfs['to_transform'][i][0].lower() or 'высшая школа экономики' in transfs['to_transform'][i][0].lower() or 'ниу вше' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'Национальный исследовательский университет - Высшая Школа Экономики'
            transfs['id'][i] = '421'

#мгу
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'ломоносов' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'Московский государственный университет им. М.В. Ломоносова'
            transfs['id'][i] = '2541'

#школа
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'общеобразовательная' in transfs['to_transform'][i][0].lower() or 'средняя' in transfs['to_transform'][i][0].lower() or 'мбоу' in transfs['to_transform'][i][0].lower() or '№' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'СОШ (Школа)'
            transfs['id'][i] = 'Sc'
#ран
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'РАН' in transfs['to_transform'][i][0] or ('наук' in transfs['to_transform'][i][0].lower() and 'академ' in transfs['to_transform'][i][0].lower() and 'рос' in transfs['to_transform'][i][0].lower()):
            transfs['new_name'][i] = 'РАН'
            transfs['id'][i] = '1432'

#мгимо
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'МГИМО' in transfs['to_transform'][i][0]:
            transfs['new_name'][i] = 'МГИМО-Университет МИД России'
            transfs['id'][i] = '901'

#юфу
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'ЮФУ' in transfs['to_transform'][i][0]:
            transfs['new_name'][i] = 'Южный федеральный университет'
            transfs['id'][i] = '322'

#спбгу
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'спбгу' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'Санкт-Петербургский государственный университет (СПбГУ)'
            transfs['id'][i] = '190'

#фин
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'финансовый' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'Финансовый Университет при Правительстве Российской Федерации'
            transfs['id'][i] = '1074'

#государство
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'правит' in transfs['to_transform'][i][0].lower() or ('совет' in transfs['to_transform'][i][0].lower() and 'фед' in transfs['to_transform'][i][0].lower()):
            transfs['new_name'][i] = 'Государственные органы'
            transfs['id'][i] = 'Gos'

#гпу им. герцена
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'герцен' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'Российский государственный педагогический университет им. А.И. Герцена'
            transfs['id'][i] = '976'

#юргпу
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'платов' in transfs['to_transform'][i][0].lower() or 'нпи' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'Южно-Российский государственный политехнический университет (НПИ) имени М.И. Платова'
            transfs['id'][i] = '633'

#СВФУ
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'аммос' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'Северо-Восточный федеральный университет им. М.К. Аммосова'
            transfs['id'][i] = '252'

#СарГУ им. Чернышевского
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'черныш' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'Саратовский государственный национальный исследовательский университет им. Н.Г. Чернышевского'
            transfs['id'][i] = '259'

#МорГУ им. Огарёва
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'огарёв' in transfs['to_transform'][i][0].lower() or 'огарев' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'Национальный исследовательский Мордовский государственный университет имени Н.П. Огарёва'
            transfs['id'][i] = '400'

#мгту баумана
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'бауман' in transfs['to_transform'][i][0].lower() or 'мгту' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'МГТУ им. Н. Э. Баумана'
            transfs['id'][i] = '665'

#БелГТУ Шухова
for i in range(len(transfs['to_transform'])):
    if transfs['id'][i] == '0':
        if 'шухов' in transfs['to_transform'][i][0].lower():
            transfs['new_name'][i] = 'Белгородский Государственный Технологический Университет им. В.Г. Шухова'
            transfs['id'][i] = '7113'

In [ ]:
# словарь значений старое название: [новое название, новый id]
trans = {}
for i in range(len(transfs)):
    trans[transfs['to_transform'][i][0]] = [[transfs['new_name'][i]], [transfs['id'][i]]]

# проходимся по всему оригинальному датафрейму, в случае, меняем значения аффилиаций и id если находим их в нашем датафрейме для изменений
for num in tqdm(range(1,9)):
    for i in range(len(df)):
        if df['author_{}_ru affiliation'.format(num)][i] in to_create:
            try:
                if trans[df['author_{}_ru affiliation'.format(num)][i][0]][0] != ['0']:
                    df['author_{}_affiliation_id'.format(num)][i] = trans[df['author_{}_ru affiliation'.format(num)][i][0]][1]
                    df['author_{}_ru affiliation'.format(num)][i] = trans[df['author_{}_ru affiliation'.format(num)][i][0]][0]
            except:
                    df['author_{}_affiliation_id'.format(num)][i] = '0'
                    print([num, i, df['author_{}_ru affiliation'.format(num)][i]])

In [ ]:
# обновляем значения измененных колонок в df
for i in affiliations:
    df[i] = af_train[i]

# Шаг 4: Создание новых ID для авторов

In [ ]:
def create_id(df, num):
  '''
  Эта функция принимает на вход датафрейм и номер (можно также заменить на название колонки)
  Чтобы функция работала с любым датафреймом, достаточно заменить названия колонок в [] на соответствующие по смыслу в другой таблице
  Оригинальные названия сохранены для понимания принципа работы функции и логики замены

  df - pandas DataFrame
  num - количество повторений (количество нумерованных колонок с id, максимальное количество авторов в базе)

  Функция проводит изменения в оригинальном датафрейме
  Если вы хотите оставить его неизменным и сохранить измененный df в новую переменную, стоит разкомментировать выход функции в конце
  '''
  for i in range(1, num+1):
    inds = []
    for i in tqdm(range(len(df))):
        if df['author_{}_eng last name'.format(num)][i] != 'none':
            try:
                surname = df['author_{}_eng last name'.format(num)][i][:8]
            except len(surname) < 8:
                surname = df['author_{}_eng last name'.format(num)][i]
        elif df['author_{}_eng last name'.format(num)][i] == 'none' and df['author_{}_ru last name'.format(num)][i] != 'none':
            surname = translit(df['author_{}_ru last name'][i],
                               'ru',
                               reversed = True)
        else:
            ind = 'N/A'
            inds.append(ind)
            continue


        ind = '_'.join([str(df['author_{}_id'.format(num)][i]),
                        df['author_{}_eng initials'.format(num)][i].replace('.',''),
                        surname,
                        str(df['author_{}_affiliation_id'.format(num)][i][0]),
                      ])
        inds.append(ind)
    df['new_id_{}'.format(num)] = inds
  #return df

In [ ]:
create_id(df, 9)

In [ ]:
# объединяем все новые id в список уникальных значений
all_new_ids = []
for i in range(1,9):
    all_new_ids += cop_df['new_id_{}'.format(i)].to_list()
all_new_ids = list(set(all_new_ids))
all_new_ids.remove('N/A') #удаляем филлерное значение из списка уникальных значений id

## Шаг 3.5: Дедупликация авторов по новым ID

In [ ]:
import jellyfish

In [ ]:
distances = {}
for i in tqdm(all_new_ids):
    for j in all_new_ids:
        a = jellyfish.damerau_levenshtein_distance(i,j)
        if 0 < a <= 4: #рассматриваем только те пары, где разница между строками <= 4 символам
            distances[(i,j)] = a

In [ ]:
sort_dist = dict(sorted(distances.items(), key=lambda item: item[1]))

# в словаре ниже получаем пары строк и разницу между ними
dist_to_sort = pd.DataFrame()
dist_to_sort['vals'] = distances.keys()
dist_to_sort['diff'] = distances.values()
val1 = []
val2 = []
for i in range(len(dist_to_sort['vals'])):
    dist_to_sort['vals'][i] = sorted(list(dist_to_sort['vals'][i]))
    val1.append(dist_to_sort['vals'][i][0])
    val2.append(dist_to_sort['vals'][i][1])
dist_to_sort['val1'] = val1
dist_to_sort['val2'] = val2